# Armis Data Hack Challenge - Solution Example

In this example we will detect anomalies on each network using Elliptic Envelope Algorithm. As this is only an example, our feature set will consist of only five features: unique hosts count, unique port count and total packets transferred. <br>
Let's start :)

# Imports and Consts

In [1]:
import pandas as pd
import datetime

In [2]:
DEVICES_PATH = "all_devices.csv"
SESSIONS_PATH = "all_sessions.csv"

## Read the data

In [3]:
devices = pd.read_csv(DEVICES_PATH)
devices.head()

FileNotFoundError: [Errno 2] File b'all_devices.csv' does not exist: b'all_devices.csv'

In [ ]:
sessions = pd.read_csv(SESSIONS_PATH)
sessions.head()

# Generate Feature Set

As we mentioned before we are going to select only 5 features: unique hosts count, unique port count,  total packets transferred, total inbound bytes and total outbound bytes. Lets create the feature set for each network

In [ ]:
def extract_features(df):
    return df.groupby(["device_id"]).aggregate({"host": "nunique", "port_dst": "nunique", 
                                                "packets_count": "sum", "inbound_bytes_count": "sum", 
                                                "outbound_bytes_count": "sum"})


In [ ]:
networks_fs = []

In [ ]:
for i in range(4):
    networks_fs.append(extract_features(sessions[sessions.network_id == i]))

# Modeling Fun

As we've mentioned we used the Elliptic Envelope model for our anomaly detection. We use the decision_function in order to receive the confidence and we normalize the values to values between 0 and 1.

In [ ]:
from sklearn.covariance import EllipticEnvelope

In [ ]:
# We use the simple min-max normalization in order to normalize the confidence values to 0-1 range.
# Higher score means that this device is probably more anomalous.
def calc_normalized_decision(decision_function_result):
    decision_function_result = -1 * decision_function_result
    minimum = decision_function_result.min()
    maximum = decision_function_result.max()
    return (decision_function_result - minimum) / (maximum - minimum)

In [ ]:
def detect_anomalies(feature_set):
    ee = EllipticEnvelope(contamination=0.05).fit(feature_set.values)
    decision_function_result = ee.decision_function(feature_set.values)
    feature_set["confidence"] = calc_normalized_decision(decision_function_result)

In [4]:
for i in range(4):
    detect_anomalies(networks_fs[i])

NameError: name 'detect_anomalies' is not defined

In [12]:
# Remove the device_id from index and Add network id to each data set
for i in range(4):
    networks_fs[i].reset_index(level=0, inplace=True)
    networks_fs[i]["network_id"] = 0

In [13]:
df_to_submit = pd.concat(networks_fs)[["network_id", "device_id", "confidence"]] # The column order is important!
df_to_submit.head()

,network_id,device_id,confidence
0,0,33,2.149737e-11
1,0,35,3.652858e-05
2,0,40,1.006346e-02
3,0,41,2.479184e-10
4,0,53,3.912391e-11


# Submissions

In order to update the Leader Board you need to send a POST request to following url: "https://leaderboard.datahack.org.il/armis/api/". Our Leader Board receives your anomalies results in the json form of [["network_id", "device_id", "confidence"]] - The order is important!

In [14]:
arr_to_submit = df_to_submit.to_json(orient='values')

In [15]:
from urllib import request
import json

leaderboard_name = "armis"
host = "leaderboard.datahack.org.il"

# Name of the user
submitter = "Armis-test"

predictions = json.loads(arr_to_submit)

jsonStr = json.dumps({'submitter': submitter, 'predictions': predictions})
data = jsonStr.encode('utf-8')
req = request.Request(f"https://{host}/{leaderboard_name}/api/",
                      headers={'Content-Type': 'application/json'},
                      data=data)
resp = request.urlopen(req)
print(json.load(resp))

{'member': 'Armis-test', 'rank': 2, 'score': 0.5648161775508931}
